# Проект урока 2. «Аналитика бронирования отелей»

## Шаг 1

Описание колонок:
* Hotel – тип отеля (City Hotel или Resort Hotel)  
* Is canceled – бронирование было отменено (1) или нет (0); не отмененное считается успешным
* Lead time – количество дней, прошедших между датой бронирования и датой прибытия  
* Arrival full date – полная дата прибытия
* Arrival date year – год прибытия  
* Arrival date month – месяц прибытия  
* Arrival date week number – номер недели прибытия
* Arrival date day of month – день прибытия
* Stays in weekend nights – количество выходных (суббота или воскресенье), которые гость забронировал для проживания в отеле
* Stays in week nights – количество дней (с понедельника по пятницу), которые гость забронировал для проживания в отеле
* Stays total nights – общее число забронированных ночей (сумма двух предыдущих колонок)
* Adults – число взрослых
* Children – число детей
* Babies – число младенцев
* Meal – выбранный тип питания
* Country – страна происхождения клиента
* Reserved room type – тип зарезервированного номера
* Assigned room type – тип полученного номера (может отличаться от забронированного)
* Customer type – тип бронирования
* Reservation status – значение последнего статуса брони: Canceled — было отменено клиентом; Check-Out — клиент зарегистрировался, но уже покинул отель; No-Show — клиент не зарегистрировался и сообщил администрации отеля причину
* Reservation status date – дата обновления статуса

In [1]:
import pandas as pd

In [2]:
bookings = pd.read_csv('2_lesson_bookings.csv', sep=';')

In [3]:
bookings.head()['Country']

0    PRT
1    PRT
2    GBR
3    GBR
4    GBR
Name: Country, dtype: object

In [4]:
# посмотрим на размер датафрейма
bookings.shape

(119390, 21)

In [5]:
# проверим типы данных 
bookings.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

In [6]:
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Hotel                      119390 non-null  object 
 1   Is Canceled                119390 non-null  int64  
 2   Lead Time                  119390 non-null  int64  
 3   arrival full date          119390 non-null  object 
 4   Arrival Date Year          119390 non-null  int64  
 5   Arrival Date Month         119390 non-null  object 
 6   Arrival Date Week Number   119390 non-null  int64  
 7   Arrival Date Day of Month  119390 non-null  int64  
 8   Stays in Weekend nights    119390 non-null  int64  
 9   Stays in week nights       119390 non-null  int64  
 10  stays total nights         119390 non-null  int64  
 11  Adults                     119390 non-null  int64  
 12  Children                   119386 non-null  float64
 13  Babies                     11

Названия колонок записаны в неудобном для дальнейшей работы формате. Исправим ситуацию, приведя названия столбцов к нижнему регистру и заменив пробелы на знак нижнего подчеркивания.

In [7]:
# пример применения
'Is Canceled'.replace(' ','_').lower()

'is_canceled'

In [8]:
# автоматизируем решение
dictionare_name = {}
for old_name in bookings.columns:
    new_name = old_name.replace(' ','_').lower()
    dictionare_name[old_name] = new_name

In [9]:
bookings = bookings.rename(columns=dictionare_name)

In [11]:
bookings.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

**Перейдем к исследованию данных.**

In [12]:
# выясним, пользователи из каких стран совершили наибольшее число успешных бронирований
# считать успешным бронированием будем то, которое в дальнейшем не было отменено 

bookings.query('is_canceled == 0').country.value_counts().head()

PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: country, dtype: int64

In [13]:
# сравним средние значения длительности бронирования в зависимости от отеля 
bookings.groupby('hotel').stays_total_nights.mean().round(2)

hotel
City Hotel      2.98
Resort Hotel    4.32
Name: stays_total_nights, dtype: float64

In [14]:
# посчитаем количество случаев овербукинга 
bookings.query('assigned_room_type != reserved_room_type').shape

(14917, 21)

In [15]:
# определим сезонность бронирования 
bookings.groupby('arrival_date_year').arrival_date_month.value_counts()

arrival_date_year  arrival_date_month
2015               September             5114
                   October               4957
                   August                3889
                   December              2920
                   July                  2776
                   November              2340
2016               October               6203
                   May                   5478
                   April                 5428
                   September             5394
                   June                  5292
                   August                5063
                   March                 4824
                   July                  4572
                   November              4454
                   February              3891
                   December              3860
                   January               2248
2017               May                   6313
                   April                 5661
                   June                  5

In [ ]:
# самый популярный месяц в 2016 - октябрь, а в 2017 - май 

In [16]:
# Выясним, в каком месяце каждого года бронирования отеля типа City Hotel отменялись чаще всего
bookings.query('hotel == "City Hotel" and is_canceled == 1') \
    .groupby('arrival_date_year') \
    .arrival_date_month \
    .value_counts()

arrival_date_year  arrival_date_month
2015               September             1543
                   October               1321
                   August                1232
                   July                   939
                   December               668
                   November               301
2016               October               1947
                   June                  1720
                   September             1567
                   April                 1539
                   May                   1436
                   November              1360
                   August                1247
                   March                 1108
                   December              1072
                   July                  1043
                   February               930
                   January                438
2017               May                   2217
                   April                 1926
                   June                  1

In [17]:
# посмотрим на средние значения количества наших клиентов относительно возраста 
bookings[['adults', 'children', 'babies']].mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

In [18]:
# для удобства объединим детей разного возраста в одну колонку 
bookings['total_kids'] =  bookings.children + bookings.babies

In [19]:
# проверим средние значения количества детей относительно типа отеля 
bookings.groupby('hotel').agg({'total_kids':'mean'}).round(2)

,total_kids
hotel,
City Hotel,0.10
Resort Hotel,0.14


Не все бронирования завершились успешно, поэтому попробуем посчитать, сколько клиентов было потеряно в процессе. Посчитаем метрику Churn Rate.

**Churn rate** (отток, коэффициент оттока) – это процент подписчиков, которые отписались от канала коммуникации, отказались от услуг сервиса в течение определенного периода времени. Т.е. представляет собой отношение количества ушедших пользователей к общему количеству пользователей, выраженное в процентах. Если Churn rate считают для какой-то определенной группы, то и количество ушедших пользователей, и общее количество пользователей считают только по этой группе.

В нашем случае Churn Rate — это процент клиентов, которые отменили бронирование. Посмотрим, как эта метрика связана с наличием детей у клиентов. **Посчитаем Churn Rate дважды: для клиентов с детьми и для клиентов без детей.** 

In [20]:
bookings['has_kids'] = bookings['total_kids'] >= 1

In [21]:
# Churn Rate клиентов без детей
round(bookings.query('has_kids==False and is_canceled==1').shape[0] / bookings.query('has_kids==False').shape[0] * 100, 2)

37.22

In [22]:
# Churn Rate клиентов с детьми
round(bookings.query('has_kids==True and is_canceled==1').shape[0] / bookings.query('has_kids==True').shape[0] * 100, 2)

34.92

## Итоги

В этом проекте я проанализировала данные о бронированиях и закрепила навыки работы с библиотекой pandas, а также познакомились с метрикой Churn Rate.